# 青空文庫データクレンジングv2

In [8]:
import os
import glob
import shutil
from zipfile import ZipFile
import numpy as np
import pandas as pd
from IPython.display import HTML
from pathlib import Path
import matplotlib.pyplot as plt

## 定数定義

In [2]:
import sys

print(type(sys.modules))
moduleList = sys.modules
ENV_COLAB = False

if 'google.colab' in moduleList:
    print("google_colab")
    ENV_COLAB = True
else:
    print("Not google_colab")

<class 'dict'>
Not google_colab


In [3]:
if ENV_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
if ENV_COLAB:
    TARGET_PATH = 'drive/MyDrive/data/target.csv'
else:
    TARGET_PATH = 'data/target.csv'

In [5]:
import pathlib

abs_path = pathlib.Path(os.getcwd())
rel_path = abs_path.relative_to(os.getcwd())
abs_path, rel_path

(PosixPath('/projects/aozora-lab'), PosixPath('.'))

In [6]:
# CARD_DIR = f'{os.getcwd()}/bunko/cards'
# CARD_DIR
CARD_DIR = f'{rel_path}/bunko/cards'
CARD_DIR

'./bunko/cards'

In [7]:
DEBUG = True

## 関数定義

In [9]:
def debug(message, flg=True):
    if flg:
        print(message)

In [10]:
def get_parameters(data):
    url = data['テキストファイルURL'].split('/')
    file_name = url[-1].split('.')[0] + '.txt'
    author_id = url[4]
    return author_id, file_name

In [11]:
def check_directory(data, card_dir):
    url = data['テキストファイルURL'].split('/')
    try:
        target_dir = f"{card_dir}/{'/'.join(url[4:-1])}"
        edit_dir = f"{target_dir}/edit"
        org_dir = f"{target_dir}/org"
        score_dir = f"{target_dir}/score"
        # files directory
        if os.path.isdir(target_dir):
            debug(f'dirs:{target_dir} already exists', DEBUG)
        else:
            debug(f'make dirs:{target_dir}', DEBUG)
            os.makedirs(target_dir)
        # files/edit directory
        if os.path.isdir(edit_dir):
            debug(f'dirs:{edit_dir} already exists', DEBUG)
        else:
            debug(f'make dirs:{edit_dir}', DEBUG)
            os.makedirs(edit_dir)
        # files/org directory
        if os.path.isdir(org_dir):
            debug(f'dirs:{org_dir} already exists', DEBUG)
        else:
            debug(f'make dirs:{org_dir}', DEBUG)
            os.makedirs(org_dir)
        # files/score directory
        if os.path.isdir(score_dir):
            debug(f'dirs:{score_dir} already exists', DEBUG)
        else:
            debug(f'make dirs:{score_dir}', DEBUG)
            os.makedirs(score_dir)

        return target_dir, edit_dir, org_dir, score_dir
    except Exception as e:
        print(e)
        return None

In [12]:
import urllib.request

# zipファイルを保存する
def save_zip(target_file, zip_file):
    try:
        file_path = ''
        if os.path.isfile(zip_file):
            file_path = zip_file
        else:
            file_path = zip_file
            urllib.request.urlretrieve(target_file, zip_file)
    except Exception as e:
        file_path = target_file
        print(e)
    finally:
        return file_path

In [13]:
# クレンジングされたデータを保存する
def save_text(target_file, org_dir, file_name, sep=','):
    try:
        save_file = f'{org_dir}/{file_name}'
        if os.path.isfile(save_file):
            # Txtファイルの読み込み
            debug(f'read {save_file}', DEBUG)
            df_org = pd.read_csv(save_file)
            pass
        else:
            # Zipファイルの読み込み
            debug(f'read {target_file}', DEBUG)
            # Zipファイル内に複数ファイルがある場合の対応
            for file in ZipFile(target_file).namelist():
                if file.endswith('.txt'):
                    df_org = pd.read_csv(ZipFile(target_file).open(file), 
                                         encoding='cp932', names=['text'], sep=sep)
                    break
            # df_org = pd.read_csv(target_file, encoding='cp932', names=['text'], sep=sep)
            df_org.to_csv(save_file, index=False)
        return df_org
    except Exception as e:
        print(f'ERROR: {target_file}, {str(e)}')
        return None

In [14]:
def get_text_body(target_df, author_name):
    # 本文の先頭を探す（'---…'区切りの直後から本文が始まる前提）
    head_txt = list(target_df[target_df['text'].str.contains(
        '-------------------------------------------------------')].index)
    # 本文の末尾を探す（'底本：'の直前に本文が終わる前提）
    atx = list(target_df[target_df['text'].str.contains('底本：')].index)
    if head_txt == []:
        # もし'---…'区切りが無い場合は、作家名の直後に本文が始まる前提
        head_txt = list(target_df[target_df['text'].str.contains(author_name)].index)
        head_txt_num = head_txt[0]+1
    else:
        # 2個目の'---…'区切り直後から本文が始まる
        head_txt_num = head_txt[1]+1
    return target_df[head_txt_num:atx[0]]

In [15]:
def split_kuten(target_df, split_kuten=True):
    df = target_df.copy()
    # 句点で分割
    if split_kuten:
        df = target_df.assign(text=target_df['text'].str.split(r'(?<=。)(?=..)')).explode('text')
    return df

In [16]:
def reset_index(target_df):
    df = target_df.copy()
    # インデックスがずれるので振りなおす
    df = df.reset_index().drop(['index'], axis=1)

    # 空白行を削除する（念のため）
    df = df[~(df['text'] == '')]

    # インデックスがずれるので振り直し、文字の長さの列を削除する
    df = df.reset_index().drop(['index'], axis=1)
    return df

In [17]:
def remove_aozora_format(target_df):
    df = target_df.copy()
    # 青空文庫の書式削除
    df = df.replace({'text': {'《.*?》': ''}}, regex=True)
    df = df.replace({'text': {'［.*?］': ''}}, regex=True)
    df = df.replace({'text': {'｜': ''}}, regex=True)
    # 字下げ（行頭の全角スペース）を削除
    df = df.replace({'text': {'　': ''}}, regex=True)
    # 節区切りを削除
    # df = df.replace({'text': {'^.$': ''}}, regex=True) # 1文字
    # df = df.replace({'text': {'^―――.*$': ''}}, regex=True) # 先頭が"―――"
    df = df.replace({'text': {'^＊＊＊.*$': ''}}, regex=True)
    df = df.replace({'text': {'^×××.*$': ''}}, regex=True)
    # # 記号、および記号削除によって残ったカッコを削除
    # df = df.replace({'text': {'―': ''}}, regex=True)
    # df = df.replace({'text': {'…': ''}}, regex=True)
    # df = df.replace({'text': {'※': ''}}, regex=True)
    df = df.replace({'text': {'「」': ''}}, regex=True)
    # 一文字以下で構成されている行を削除 -> しない
    # df['length'] = df['text'].map(lambda x: len(x))
    # df = df[df['length'] > 1]
    return df

In [18]:
def add_type_column(target_df):
    df = target_df.copy()
    df.insert(loc=0, column='type', value='本文')
    df.loc[df['text'].str.contains('字下げ.*見出し*'), 'type'] = '見出し'
    return df

## 前処理

In [79]:
target_list = pd.read_csv(TARGET_PATH, index_col=0)
target_list.shape

(3442, 13)

In [80]:
print('----- 以下は対象外とする -> テキストを前処理できるように修正した -----')
print(target_list.loc[3256])
target_list.loc[3256, '対象'] = True
target_list.loc[3256, '備考'] = 'テキストを前処理できるように修正した'
print('-----')
print(target_list.loc[3259])
target_list.loc[3259, '対象'] = True
target_list.loc[3259, '備考'] = 'テキストを前処理できるように修正した'

----- 以下は対象外とする -> テキストを前処理できるように修正した -----
対象                                                         False
人物ID                                                         168
氏名                                                         横光 利一
読み                                                      よこみつ りいち
作品ID                                                         906
作品名                                                           時間
副題                                                           NaN
作品名読み                                                        じかん
図書カードURL       https://www.aozora.gr.jp/cards/000168/card906....
テキストファイルURL    https://www.aozora.gr.jp/cards/000168/files/90...
テキストファイルパス                                                   NaN
備考                                                   例外起きる&行数少ない
length                                                       NaN
Name: 3256, dtype: object
-----
対象                                                         False
人物ID          

In [81]:
# エラーが起きるもの等は対象をFalseにして対象外とする
target_list[target_list['対象']==False]

,対象,人物ID,氏名,読み,作品ID,作品名,副題,作品名読み,図書カードURL,テキストファイルURL,テキストファイルパス,備考,length


In [82]:
target_list[target_list['対象']==True].shape

(3442, 13)

In [ ]:
import time

DEBUG = False
try:
    total = target_list[target_list['対象']==True].shape[0]
    for i, target_data in target_list[target_list['対象']==True].iterrows():
        # 必要な情報の取得
        author_id, file_name = get_parameters(target_data)
        target_dir, edit_dir, org_dir, score_dir = check_directory(target_data, CARD_DIR)
        target_file = target_data['テキストファイルURL']
        zip_file = f"{target_dir}/{target_file.split('/')[-1]}" # add
        author_name = target_data['氏名'].replace(' ', '')
        novel_name = target_data['作品名']
        edit_file = f'{edit_dir}/{file_name}'
        debug(f'--- {str(i).zfill(4)}/{str(total).zfill(4)} {author_id} {author_name} {novel_name} ---', True)
        # データクレンジング実施
        file_path = save_zip(target_file, zip_file)
        target_df = save_text(file_path, org_dir, file_name) # データ取得＆保存
        if target_df is None:
            debug(f'--> save_text error: {novel_name} <--', DEBUG)
            target_df = save_text(file_path, org_dir, file_name, '@')

        if target_df is None:
            debug(f'--> skip {novel_name} <--', DEBUG)
        elif os.path.isfile(edit_file):
            debug(f'{novel_name} has been already cleansed', DEBUG)
            target_list.loc[i,'テキストファイルパス'] = edit_file
            target_list.loc[i,'length'] = len(pd.read_csv(edit_file))
        else:
            body_df = get_text_body(target_df, author_name) # 本文取得
            body_df2 = split_kuten(body_df) # 句点「。」で分割
            body_df3 = add_type_column(body_df2) # type列追加
            body_df4 = remove_aozora_format(body_df3) # 青空文庫の書式を削除
            body_df5 = reset_index(body_df4) # インデックスをリセット
            # 結果の保存
            body_df5.to_csv(edit_file, index=False)
            target_list.loc[i,'テキストファイルパス'] = edit_file
            target_list.loc[i,'length'] = body_df5.shape[0]
            debug(f'length: {body_df5.shape[0]}', DEBUG)
        time.sleep(1.0) # zip取得する際の負荷回避
        if i > 10:      # テスト用:全部取得する場合はコメントアウト
            break
except Exception as e:
    print(target_data)
    print(e)


In [89]:
# リストをCSVに出力する
target_list.to_csv('data/target.csv')

In [90]:
target_list[target_list['length']>500]

,対象,人物ID,氏名,読み,作品ID,作品名,副題,作品名読み,図書カードURL,テキストファイルURL,テキストファイルパス,備考,length
13,True,879,芥川 竜之介,あくたがわ りゅうのすけ,124,お律と子等と,NaN,おりつとこらと,https://www.aozora.gr.jp/cards/000879/card124....,https://www.aozora.gr.jp/cards/000879/files/12...,./bunko/cards/000879/files/edit/124_ruby_952.txt,NaN,982.0
19,True,879,芥川 竜之介,あくたがわ りゅうのすけ,69,河童,NaN,かっぱ,https://www.aozora.gr.jp/cards/000879/card69.html,https://www.aozora.gr.jp/cards/000879/files/69...,./bunko/cards/000879/files/edit/69_ruby_1321.txt,NaN,1228.0
24,True,879,芥川 竜之介,あくたがわ りゅうのすけ,77,奇怪な再会,NaN,きかいなさいかい,https://www.aozora.gr.jp/cards/000879/card77.html,https://www.aozora.gr.jp/cards/000879/files/77...,./bunko/cards/000879/files/edit/77_ruby_944.txt,NaN,727.0
30,True,879,芥川 竜之介,あくたがわ りゅうのすけ,38,戯作三昧,NaN,げさくざんまい,https://www.aozora.gr.jp/cards/000879/card38.html,https://www.aozora.gr.jp/cards/000879/files/38...,./bunko/cards/000879/files/edit/38_ruby_3638.txt,NaN,662.0
41,True,879,芥川 竜之介,あくたがわ りゅうのすけ,59,邪宗門,NaN,じゃしゅうもん,https://www.aozora.gr.jp/cards/000879/card59.html,https://www.aozora.gr.jp/cards/000879/files/59...,./bunko/cards/000879/files/edit/59_ruby_849.txt,NaN,790.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3376,True,1562,吉川 英治,よしかわ えいじ,56053,山浦清麿,NaN,やまうらきよまろ,https://www.aozora.gr.jp/cards/001562/card5605...,https://www.aozora.gr.jp/cards/001562/files/56...,./bunko/cards/001562/files/edit/56053_ruby_529...,NaN,1978.0
3377,True,1562,吉川 英治,よしかわ えいじ,56052,夕顔の門,NaN,ゆうがおのもん,https://www.aozora.gr.jp/cards/001562/card5605...,https://www.aozora.gr.jp/cards/001562/files/56...,./bunko/cards/001562/files/edit/56052_ruby_529...,NaN,617.0
3378,True,1562,吉川 英治,よしかわ えいじ,56046,牢獄の花嫁,NaN,ろうごくのはなよめ,https://www.aozora.gr.jp/cards/001562/card5604...,https://www.aozora.gr.jp/cards/001562/files/56...,./bunko/cards/001562/files/edit/56046_ruby_604...,NaN,6379.0
3411,True,325,蘭 郁二郎,らん いくじろう,2187,夢鬼,NaN,むき,https://www.aozora.gr.jp/cards/000325/card2187...,https://www.aozora.gr.jp/cards/000325/files/21...,./bunko/cards/000325/files/edit/2187_ruby_3915...,NaN,1792.0


## Step by Step

In [20]:
import pathlib

abs_path = pathlib.Path(os.getcwd())
rel_path = abs_path.relative_to(os.getcwd())
abs_path, rel_path

(PosixPath('/projects/aozora-lab'), PosixPath('.'))

In [21]:
CARD_DIR = f'{rel_path}/bunko/cards'
CARD_DIR

'./bunko/cards'

- 2200: 草枕
- 2202: 行人
- 1887: 田舎教師

In [40]:
target_data = target_list.loc[3259]
target_data

Unnamed: 0                                                  3259
対象                                                         False
人物ID                                                         168
氏名                                                         横光 利一
読み                                                      よこみつ りいち
作品ID                                                         909
作品名                                                            鳥
副題                                                           NaN
作品名読み                                                         とり
図書カードURL       https://www.aozora.gr.jp/cards/000168/card909....
テキストファイルURL    https://www.aozora.gr.jp/cards/000168/files/90...
テキストファイルパス                                                   NaN
備考                                                   例外起きる&行数少ない
length                                                       NaN
Name: 3259, dtype: object

In [41]:
author_id, file_name = get_parameters(target_data)
print(f'author_id:{author_id}')
print(f'file_name:{file_name}')

author_id:000168
file_name:909_ruby_518.txt


In [42]:
target_dir, edit_dir, org_dir, score_dir = check_directory(target_data, CARD_DIR)

print(f'target_dir:{target_dir}')
print(f'edit_dir:{edit_dir}')
print(f'org_dir:{org_dir}')
print(f'score_dir:{score_dir}')

dirs:./bunko/cards/000168/files already exists
dirs:./bunko/cards/000168/files/edit already exists
dirs:./bunko/cards/000168/files/org already exists
dirs:./bunko/cards/000168/files/score already exists
target_dir:./bunko/cards/000168/files
edit_dir:./bunko/cards/000168/files/edit
org_dir:./bunko/cards/000168/files/org
score_dir:./bunko/cards/000168/files/score


In [43]:
target_file = target_data['テキストファイルURL']
author_name = target_data['氏名'].replace(' ', '')
zip_file = f"{target_dir}/{target_file.split('/')[-1]}" # add
print(f'target_file: {target_file}')
print(f'author_name: {author_name}')
print(f'zip_file: {zip_file}')

target_file: https://www.aozora.gr.jp/cards/000168/files/909_ruby_518.zip
author_name: 横光利一
zip_file: ./bunko/cards/000168/files/909_ruby_518.zip


In [44]:
file_path = save_zip(target_file, zip_file)
file_path

'./bunko/cards/000168/files/909_ruby_518.zip'

In [45]:
# データ取得＆保存
target_df = save_text(file_path, org_dir, file_name) # 本文に[,]があるとエラーになる
if target_df is None:
    target_df = save_text(file_path, org_dir, file_name, '@') # 区切りを[@]にしておく
target_df.head()

read ./bunko/cards/000168/files/org/909_ruby_518.txt


,text
0,鳥
1,横光利一
2,----------------------------------------------...
3,【テキスト中に現れる記号について】
4,《》：ルビ


In [46]:
# 本文取得
body_df = get_text_body(target_df, author_name)
body_df.head()

,text
12,リカ子《こ》はときどき私《わたし》の顔《かお》を盗見《ぬすみみ》するように艶《つや》のある...


In [47]:
# 句点「。」で分割
body_df2 = split_kuten(body_df)
body_df2.head()

,text
12,リカ子《こ》はときどき私《わたし》の顔《かお》を盗見《ぬすみみ》するように艶《つや》のある...
12,私《わたし》は彼女《かのじょ》が何《な》ぜそんな顔《かお》を今日《きょう》に限《かぎ》ってす...
12,便利《べんり》なことには私《わたし》はリカ子《こ》を彼女《かのじょ》の良人《おっと》から奪《...
12,何《な》ぜなら私《わたし》はリカ子《こ》を彼女《かのじょ》の良人《おっと》に奪《うば》われた...
12,この不幸《ふこう》なことが幸《さいわ》いにも今頃《いまごろ》幸福《こうふく》な結果《けっか》...


In [48]:
# type列追加
body_df3 = add_type_column(body_df2)
body_df3.head()

,type,text
12,本文,リカ子《こ》はときどき私《わたし》の顔《かお》を盗見《ぬすみみ》するように艶《つや》のある...
12,本文,私《わたし》は彼女《かのじょ》が何《な》ぜそんな顔《かお》を今日《きょう》に限《かぎ》ってす...
12,本文,便利《べんり》なことには私《わたし》はリカ子《こ》を彼女《かのじょ》の良人《おっと》から奪《...
12,本文,何《な》ぜなら私《わたし》はリカ子《こ》を彼女《かのじょ》の良人《おっと》に奪《うば》われた...
12,本文,この不幸《ふこう》なことが幸《さいわ》いにも今頃《いまごろ》幸福《こうふく》な結果《けっか》...


In [49]:
# 青空文庫の書式を削除
body_df4 = remove_aozora_format(body_df3)
body_df4.head()

,type,text
12,本文,リカ子はときどき私の顔を盗見するように艶のある眼を上げた。
12,本文,私は彼女が何ぜそんな顔を今日に限ってするのか初めの間は見当がつかなかったのだが、それが分った...
12,本文,便利なことには私はリカ子を彼女の良人から奪おうという気もなければ彼女を奪う必要もないことだ。
12,本文,何ぜなら私はリカ子を彼女の良人に奪われたのだからである。
12,本文,この不幸なことが幸いにも今頃幸福な結果になって来たということは、私にとっては依然として不幸な...


In [50]:
# インデックスをリセット
body_df5 = reset_index(body_df4)
body_df5.head()

,type,text
0,本文,リカ子はときどき私の顔を盗見するように艶のある眼を上げた。
1,本文,私は彼女が何ぜそんな顔を今日に限ってするのか初めの間は見当がつかなかったのだが、それが分った...
2,本文,便利なことには私はリカ子を彼女の良人から奪おうという気もなければ彼女を奪う必要もないことだ。
3,本文,何ぜなら私はリカ子を彼女の良人に奪われたのだからである。
4,本文,この不幸なことが幸いにも今頃幸福な結果になって来たということは、私にとっては依然として不幸な...


In [51]:
body_df5.to_csv(f'{edit_dir}/{file_name}', index=False)

## チェック

In [19]:
target_file = pd.read_csv('data/target.csv', index_col=0)
target_file.shape

(3442, 13)

In [95]:
target_file.columns

Index(['対象', '人物ID', '氏名', '読み', '作品ID', '作品名', '副題', '作品名読み', '図書カードURL',
       'テキストファイルURL', 'テキストファイルパス', '備考', 'length'],
      dtype='object')

In [96]:
# テキストファイルパスで指定されたパスにファイルが存在するか
count = 0
for id, row in target_list.iterrows():
    f = row['テキストファイルパス']
    if type(f)==float or not os.path.isfile(f):
        print(id, row['氏名'], row['作品ID'], row['作品名'], row['テキストファイルURL'])
        count += 1
print('count: ', count)

count:  0


In [13]:
target_old = pd.read_excel('untrack/target.xlsx', index_col=0)
target_old.head()

,対象,人物ID,氏名,読み,作品ID,作品名,副題,作品名読み,図書カードURL,テキストファイルURL,テキストファイルパス,備考,length
id,,,,,,,,,,,,,
0,True,374,饗庭 篁村,あえば こうそん,45754,良夜,NaN,りょうや,https://www.aozora.gr.jp/cards/000374/card4575...,https://www.aozora.gr.jp/cards/000374/files/45...,./bunko/cards/000374/files/edit/45754_ruby_236...,NaN,119.0
1,True,879,芥川 竜之介,あくたがわ りゅうのすけ,73,或敵打の話,NaN,あるかたきうちのはなし,https://www.aozora.gr.jp/cards/000879/card73.html,https://www.aozora.gr.jp/cards/000879/files/73...,./bunko/cards/000879/files/edit/73_ruby_1217.txt,NaN,280.0
2,True,879,芥川 竜之介,あくたがわ りゅうのすけ,122,或日の大石内蔵助,NaN,あるひのおおいしくらのすけ,https://www.aozora.gr.jp/cards/000879/card122....,https://www.aozora.gr.jp/cards/000879/files/12...,./bunko/cards/000879/files/edit/122_ruby_172.txt,NaN,214.0
3,True,879,芥川 竜之介,あくたがわ りゅうのすけ,129,或恋愛小説,NaN,あるれんあいしょうせつ,https://www.aozora.gr.jp/cards/000879/card129....,https://www.aozora.gr.jp/cards/000879/files/12...,./bunko/cards/000879/files/edit/129_ruby_1142.txt,NaN,182.0
4,True,879,芥川 竜之介,あくたがわ りゅうのすけ,57,一夕話,NaN,いっせきわ,https://www.aozora.gr.jp/cards/000879/card57.html,https://www.aozora.gr.jp/cards/000879/files/57...,./bunko/cards/000879/files/edit/57_ruby_1172.txt,NaN,209.0


In [48]:
print(target_old.shape)
print(target_file.shape)

(3442, 13)
(3442, 13)


In [30]:
old500 = target_old[target_old['length']>=500]

In [31]:
new500 = target_file[target_file['length']>=500]

In [81]:
merge_target = pd.merge(old500, new500, how='right', left_index=True, right_index=True)
merge_length = merge_target.loc[:,['作品名_x', '作品名_y', 'length_x', 'length_y']]
merge_length.head()

,作品名_x,作品名_y,length_x,length_y
13,お律と子等と,お律と子等と,982.0,982.0
19,NaN,河童,NaN,1228.0
24,奇怪な再会,奇怪な再会,727.0,727.0
30,戯作三昧,戯作三昧,662.0,662.0
41,邪宗門,邪宗門,790.0,790.0


In [82]:
merge_length.shape

(1149, 4)

In [73]:
count = 0
for i in range(len(merge_length)):
    d = merge_length.iloc[i]
    if d['length_y']!=d['length_y']:
        count += 1
        print(d['作品名_x'], d['作品名_y'], d['length_x'], d['length_y'])
print(f'count:{count}')

count:0


## チェック2

In [9]:
path = 'bunko/cards/000479/files/org/49198_ruby_32155.txt'
df = pd.read_csv(path)
# 中央揃えを実現するStylerクラスのインスタンスを作成
styled = df.style.set_properties(**{'text-align': 'left'})
 
# HTML形式に変換して表示
HTML(styled.render())

/tmp/ipykernel_1103692/3727791846.py:7: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  HTML(styled.render())


,text
0,今戸狐
1,小山内薫
2,-------------------------------------------------------
3,【テキスト中に現れる記号について】
4,《》：ルビ
5,（例）矢張《やっぱり》
6,｜：ルビの付く文字列の始まりを特定する記号
7,（例）晩｜他所《よそ》からの
8,-------------------------------------------------------
9,これは狐か狸だろう、矢張《やっぱり》、俳優《やくしゃ》だが、数年《すねん》以前のこと、今の沢村宗十郎《さわむらそうじゅうろう》氏の門弟で某《なにがし》という男が、或《ある》夏の晩｜他所《よそ》からの帰りが大分遅くなったので、折詰を片手にしながら、てくてく馬道《うまみち》の通りを急いでやって来て、さて聖天《しょうてん》下の今戸橋《いまどばし》のところまで来ると、四辺《あたり》は一面の出水《でみず》で、最早《もはや》如何《どう》することも出来ない、車屋と思ったが、あたりには、人の影もない、橋の上も一尺ばかり水が出て、濁水《だくすい》がゴーゴーという音を立てて、隅田川《すみだがわ》の方へ流込《ながれこ》んでいる、致方《しかた》がないので、衣服《きもの》の裾《すそ》を、思うさま絡上《まくりあ》げて、何しろこの急流｜故《ゆえ》、流されては一大事と、犬の様に四這《よつんばい》になって、折詰は口に銜《くわ》えながら無我夢中、一生懸命になって、「危険《あぶない》危険《あぶない》」と自分で叫びながら、漸《ようや》く、向うの橋詰《はしつめ》までくると、其処《そこ》に白い着物を着た男が、一人立っていて盛《さかん》に笑っているのだ、おかしな奴だと思って不図《ふと》見ると、交番所《こうばんしょ》の前に立っていた巡査だ、巡査は笑いながら「一体《いったい》今何をしていたのか」と訊くから、何しろこんな、出水《しゅっすい》で到底《とうてい》渡れないから、こうして来たのだといいながら、ふと後《うしろ》を振返《ふりかえ》って見ると、出水《しゅっすい》どころか、道もからからに乾いて、橋の上も、平時《いつも》と少しも変りがない、おやッ、こいつは一番やられたわいと、手にした折詰を見ると、こは如何《いか》に、底は何時《いつ》しかとれて、内はからんからん、遂《つい》に大笑いをして、それからまた師匠の家《うち》へ帰っても、盛《さかん》に皆《みんな》から笑われたとの事だ。


In [7]:
path = 'bunko/cards/000479/files/edit/49198_ruby_32155.txt'
df = pd.read_csv(path)
# Stylerクラスのインスタンスを作成
styled = df.style.set_properties(**{'text-align': 'left'})
 
# HTML形式に変換して表示
HTML(styled.render())

/tmp/ipykernel_1103692/313934399.py:7: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  HTML(styled.render())


,type,text
0,本文,これは狐か狸だろう、矢張、俳優だが、数年以前のこと、今の沢村宗十郎氏の門弟で某という男が、或夏の晩他所からの帰りが大分遅くなったので、折詰を片手にしながら、てくてく馬道の通りを急いでやって来て、さて聖天下の今戸橋のところまで来ると、四辺は一面の出水で、最早如何することも出来ない、車屋と思ったが、あたりには、人の影もない、橋の上も一尺ばかり水が出て、濁水がゴーゴーという音を立てて、隅田川の方へ流込んでいる、致方がないので、衣服の裾を、思うさま絡上げて、何しろこの急流故、流されては一大事と、犬の様に四這になって、折詰は口に銜えながら無我夢中、一生懸命になって、「危険危険」と自分で叫びながら、漸く、向うの橋詰までくると、其処に白い着物を着た男が、一人立っていて盛に笑っているのだ、おかしな奴だと思って不図見ると、交番所の前に立っていた巡査だ、巡査は笑いながら「一体今何をしていたのか」と訊くから、何しろこんな、出水で到底渡れないから、こうして来たのだといいながら、ふと後を振返って見ると、出水どころか、道もからからに乾いて、橋の上も、平時と少しも変りがない、おやッ、こいつは一番やられたわいと、手にした折詰を見ると、こは如何に、底は何時しかとれて、内はからんからん、遂に大笑いをして、それからまた師匠の家へ帰っても、盛に皆から笑われたとの事だ。
